In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress
import os
from glob import glob

# === CONFIGURACIÓN ===
carpeta_entrada = r"10925"  # Cambia por tu ruta
carpeta_salida = os.path.join(carpeta_entrada, "graficas")

# Crear carpeta de salida si no existe
os.makedirs(carpeta_salida, exist_ok=True)

# Listar todos los .txt
archivos = glob(os.path.join(carpeta_entrada, "*.txt"))

print(f"Se encontraron {len(archivos)} archivos.")

# Lista para guardar resultados
resultados = []

# === PROCESAR CADA ARCHIVO ===
for archivo in archivos:
    try:
        # Leer archivo (sin headers, 9 columnas, tabulado)
        df = pd.read_csv(archivo, sep="\t", header=None)

        # Extraer columnas 2 y 4
        x = pd.to_numeric(df.iloc[:, 3], errors='coerce')
        y = pd.to_numeric(df.iloc[:, 1], errors='coerce')

        # Eliminar NaN
        mask = ~np.isnan(x) & ~np.isnan(y)
        x = x[mask].values
        y = y[mask].values

        if len(x) < 3:
            print(f"Archivo {archivo} omitido (datos insuficientes).")
            continue

        # Regresión lineal
        res = linregress(x, y)
        m = res.slope
        b = res.intercept
        SE_m = res.stderr
        SE_b = res.intercept_stderr
        r2 = res.rvalue**2

        # Guardar resultados en lista
        nombre_base = os.path.splitext(os.path.basename(archivo))[0]
        resultados.append({
            "archivo": nombre_base,
            "pendiente": m,
            "error_pendiente": SE_m,
            "intercepto": b,
            "error_intercepto": SE_b,
            "R2": r2
        })

        # Predicción para la recta
        x_pred = np.linspace(x.min(), x.max(), 100)
        y_pred = m * x_pred + b

        # === GRÁFICA ===
        plt.figure(figsize=(8, 5))
        plt.scatter(x, y, color='dodgerblue', alpha=0.7, label='Datos')
        plt.plot(x_pred, y_pred, color='red', linewidth=2, label='Regresión lineal')

        # Texto con ecuación, errores y R²
        ecuacion = (f"y = {m:.4f}x + {b:.4f}\n"
                    f"±({SE_m:.4f}, {SE_b:.4f})\n"
                    f"R² = {r2:.4f}")
        plt.text(0.02, 0.95, ecuacion, transform=plt.gca().transAxes,
                 fontsize=10, color='red', verticalalignment='top',
                 bbox=dict(facecolor='white', alpha=0.7))

        plt.xlabel('V (V)')
        plt.ylabel('I (A)')
        plt.title(f'Regresión Lineal: {nombre_base}')
        plt.legend(loc='lower right')
        plt.grid(True)

        # Guardar gráfica
        salida = os.path.join(carpeta_salida, f"{nombre_base}_regresion.png")
        plt.savefig(salida, dpi=300, bbox_inches='tight')
        plt.close()

        print(f"✅ Gráfica guardada: {salida}")

    except Exception as e:
        print(f"❌ Error procesando {archivo}: {e}")

# === GUARDAR RESUMEN EN CSV ===
df_resultados = pd.DataFrame(resultados)
csv_resumen = os.path.join(carpeta_salida, "resultados_regresiones.csv")
df_resultados.to_csv(csv_resumen, index=False)

print(f"\n📄 Archivo resumen guardado en: {csv_resumen}")
print(df_resultados.head())

Se encontraron 29 archivos.
✅ Gráfica guardada: 10925/graficas/IvsV_PY_Vg-10mV_010925_Vbias10mV_regresion.png
✅ Gráfica guardada: 10925/graficas/IvsV_PY_Vg-1V_010925_Vbias10mV_regresion.png
✅ Gráfica guardada: 10925/graficas/IvsV_PY_Vg-2V_010925_Vbias10mV_regresion.png
✅ Gráfica guardada: 10925/graficas/IvsV_PY_Vg-4V_010925_Vbias10mV_regresion.png
✅ Gráfica guardada: 10925/graficas/IvsV_PY_Vg-5V_010925_Vbias10mV_regresion.png
✅ Gráfica guardada: 10925/graficas/IvsV_PY_Vg-5V_010925_Vbias10mV_2_regresion.png
✅ Gráfica guardada: 10925/graficas/IvsV_PY_Vg-5mV_010925_Vbias10mV_regresion.png
✅ Gráfica guardada: 10925/graficas/IvsV_PY_Vg-6mV_010925_Vbias10mV_regresion.png
✅ Gráfica guardada: 10925/graficas/IvsV_PY_Vg-7mV_010925_Vbias10mV_regresion.png
✅ Gráfica guardada: 10925/graficas/IvsV_PY_Vg-8mV_010925_Vbias10mV_regresion.png
✅ Gráfica guardada: 10925/graficas/IvsV_PY_Vg-9mV_010925_Vbias10mV_regresion.png
✅ Gráfica guardada: 10925/graficas/IvsV_PY_Vg0V_010925_Vbias10mV_regresion.png
✅ Gr